In [9]:
import warnings
warnings.filterwarnings('ignore')

import geopandas as gpd
from shapely import geometry
import pandas as pd
import os

In [10]:
!ls

TX0County.json              TX7County.json
TX1County.json              TX_County.json
TX2County.json              TX_County_Cleaned.json
TX3County.json              TX_County_Parser.zip
TX4County.json              Texas_County_Boundaries
TX5County.json              Texas_County_Boundaries.zip
TX6County.json              parse_county_forDB.ipynb


In [11]:
gdf = gpd.read_file("TX_County_Cleaned.json")
gdf.head(1)

,County,geometry
0,Uvalde County,"POLYGON ((-100.11112 29.08625, -100.11122 29.1..."


In [12]:
len(gdf["County"].unique()

SyntaxError: unexpected EOF while parsing (<ipython-input-12-66e328511cdd>, line 1)

In [13]:
counties = sorted(list(gdf["County"].unique()))

In [17]:
file_name = "TX7County.json"
g = gdf[gdf["County"].isin(counties[7*31:])]
g.to_file(file_name, driver="GeoJSON")

In [16]:
gdf[gdf["County"].isin(counties[7*31:])]["County"].unique()

array(['Uvalde County', 'Wharton County', 'Val Verde County',
       'Waller County', 'Washington County', 'Travis County',
       'Terrell County', 'Sutton County', 'Williamson County',
       'Walker County', 'Tyler County', 'Trinity County',
       'Willacy County', 'Zapata County', 'Webb County', 'Zavala County',
       'Victoria County', 'Wilson County', 'Van Zandt County',
       'Upshur County', 'Upton County', 'Ward County', 'Tom Green County',
       'Winkler County', 'Taylor County', 'Wood County',
       'Throckmorton County', 'Young County', 'Wichita County',
       'Wilbarger County', 'Terry County', 'Yoakum County', 'Wise County',
       'Titus County', 'Tarrant County', 'Swisher County',
       'Wheeler County'], dtype=object)

In [10]:
254//8

31

In [6]:
g = gpd.read_file("TX0County.json")

In [8]:
g["County"].unique()

array(['Brazoria County', 'Bexar County', 'Bandera County',
       'Caldwell County', 'Austin County', 'Bastrop County',
       'Blanco County', 'Brewster County', 'Burleson County',
       'Brazos County', 'Burnet County', 'Bell County', 'Cameron County',
       'Brooks County', 'Aransas County', 'Bee County', 'Calhoun County',
       'Atascosa County', 'Andrews County', 'Borden County',
       'Angelina County', 'Brown County', 'Anderson County',
       'Bosque County', 'Callahan County', 'Bailey County',
       'Bowie County', 'Baylor County', 'Archer County', 'Briscoe County',
       'Armstrong County'], dtype=object)

# create x y for each polygon

In [9]:
Polygon = geometry.polygon.Polygon
MultiPolygon = geometry.multipolygon.MultiPolygon

In [10]:
def parse(gdf, arg_i, arg_shape_id, arg_polygon_id):
    columns = ["geovertex_id", "shape_id", "polygon_id", "County", "x_pos", "y_pos"]
    df = pd.DataFrame(columns=columns)

    counter = 0
    i = arg_i
    shape_id = arg_shape_id
    polygon_id = arg_polygon_id

    for index, row in gdf.iterrows():
        # for visualization only
        if counter%15==0:
            print(" {:.2f}% ".format(counter/len(gdf)*100), end="")
        else:
            print("#", end="")

        cur_shape = row["geometry"]
        cur_type = type(cur_shape)

        row = row.drop(["geometry"])
        cur_row = row
        cur_row["shape_id"] = shape_id
        shape_id += 1

        if cur_type == MultiPolygon:
            for polygon in cur_shape:
                cur_row["polygon_id"] = polygon_id
                polygon_id += 1
                for coords in polygon.exterior.coords:
                    cur_row["geovertex_id"] = i
                    i += 1
                    cur_row["x_pos"] = coords[0]
                    cur_row["y_pos"] = coords[1]
                    df = df.append(cur_row, ignore_index=True)
                    cur_row = row
        elif cur_type == Polygon:
            cur_row["polygon_id"] = polygon_id
            polygon_id += 1
            for coords in cur_shape.exterior.coords:
                cur_row["geovertex_id"] = i
                i += 1
                cur_row["x_pos"] = coords[0]
                cur_row["y_pos"] = coords[1]
                df = df.append(cur_row, ignore_index=True)
                cur_row = row
        counter += 1
        
    return df

In [13]:
next_i, next_shape, next_poly = 647340, 1, 719
for i in range(8):
    file_name = "TX" + str(i) + "County.json"
    
    print("\n\nStart", file_name)
    gdf = gpd.read_file(file_name)
    gdf_Parsed = parse(gdf, arg_i=next_i, arg_shape_id=next_shape, arg_polygon_id=next_poly)

    file_name = file_name.split(".")[0]
    gdf_Parsed.to_csv(file_name+"_Parsed.csv")

    next_i = gdf_Parsed.iloc[-1]["geovertex_id"] + 1
    next_shape = gdf_Parsed.iloc[-1]["shape_id"] + 1
    next_poly = gdf_Parsed.iloc[-1]["polygon_id"] + 1
    print("\nLast row id: geo:{}, shape:{}, poly:{}".format(gdf_Parsed.iloc[-1]["geovertex_id"],
                                                         gdf_Parsed.iloc[-1]["shape_id"],
                                                         gdf_Parsed.iloc[-1]["polygon_id"]))
    print("Next id: geo:{}, shape:{}, poly:{}".format(next_i, next_shape, next_poly))



Start VA0County.json
 0.00% ############## 53.57% ############
Last row id: geo:790355, shape:28, poly:751
Next id: geo:790356, shape:29, poly:752


Start VA1County.json
 0.00% ############## 53.57% ############
Last row id: geo:937599, shape:56, poly:784
Next id: geo:937600, shape:57, poly:785


Start VA2County.json
 0.00% ############## 53.57% ############
Last row id: geo:1085946, shape:84, poly:814
Next id: geo:1085947, shape:85, poly:815


Start VA3County.json
 0.00% ############## 53.57% ############
Last row id: geo:1204646, shape:112, poly:855
Next id: geo:1204647, shape:113, poly:856


In [14]:
all_dfs = []
rows = 0
for i in range(4):
    FILE_NAME = "VA" + str(i) + "County_Parsed.csv"
    all_dfs.append(pd.read_csv(FILE_NAME))
    rows += len(all_dfs[-1])

merged = pd.concat(all_dfs)
print(len(merged), rows)

557307 557307


In [15]:
merged.to_csv("VA_County_Parsed_Merged.csv")

In [4]:
#precinct = pd.read_csv("../precinct/VA_Final_Merged.csv")

In [5]:
#precinct.head(-1)

,geovertex_id,shape_id,polygon_id,x_pos,y_pos
0,419895,1,941,-76.943785,37.780534
1,419896,1,941,-76.944097,37.780237
2,419897,1,941,-76.944536,37.779986
3,419898,1,941,-76.945793,37.779563
4,419899,1,941,-76.946149,37.779332
...,...,...,...,...,...
2569558,2989453,2446,3467,-76.480168,37.236542
2569559,2989454,2446,3467,-76.479523,37.236377
2569560,2989455,2446,3467,-76.478359,37.236079
2569561,2989456,2446,3467,-76.473780,37.234929
